In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


In [12]:
# file = "sat-data.h5"
# sat_data = pd.read_hdf(file)
filex = "sat-data.npy"
filey = "sat-labels.npy"

sat_data = np.load( filex )
sat_labels = np.load( filey, allow_pickle=True )

In [13]:
sat_data = sat_data.astype('float32')
sat_labels = np.array([label[0] for label in sat_labels]).astype('int32')

In [14]:
## Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(sat_data, sat_labels, test_size=0.2, random_state=42)

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Softmax, Conv1D, MaxPooling1D

# Define the CNN model
model = Sequential()

# input_shape = (20, 6, 1)
# First convolutional layer (assuming 1D data)
model.add(Conv1D(filters=128, kernel_size=2, activation='relu'))

# Second convolutional layer
model.add(Conv1D(filters=128, kernel_size=2, activation='relu'))

# Max pooling layer
model.add(MaxPooling1D(pool_size=2))

# Flatten the output
model.add(Flatten())

# Fully connected layers
model.add(Dense(units=192, activation='relu'))
model.add(Dense(units=89, activation='relu'))

# Output layer with 3 units for maneuver classification (no maneuver, beginning of maneuver, during maneuver) not float but integer
model.add(Dense(units=3, activation='softmax'))

# # Output layer with 3 output units for classification, [1, 0, 0], [0, 1, 0], [0, 0, 1] sigmoid activation function
# model.add(Dense(units=3, activation='sigmoid'))

# Compile the model (loss function and optimizer might be different in the paper)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Implement 10-fold cross-validation (code not shown here, refer to Kera's documentation)
model.build(input_shape=sat_data.shape)

model.summary()
# Train the model (refer to Keras documentation for training with validation sets)
# model.fit(X_train, y_train, epochs=5)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)               │ (127, 4, 128)          │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (127, 3, 128)          │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (127, 1, 128)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (127, 128)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (127, 192)             │        24,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (127, 89)              │        17,177 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (127, 3)               │           270 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 76,775 (299.90 KB)

 Trainable params: 76,775 (299.90 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
sat_labels[0]

array([1, 0, 0])

In [17]:

model.fit(sat_data, sat_labels, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10


4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 80ms/step - accuracy: 0.5432 - loss: 1.0422 - val_accuracy: 0.1154 - val_loss: 1.3342
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6718 - loss: 0.7926 - val_accuracy: 0.1154 - val_loss: 1.7219
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6893 - loss: 0.6141 - val_accuracy: 0.1154 - val_loss: 2.1912
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8683 - loss: 0.4915 - val_accuracy: 0.5385 - val_loss: 2.5993
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9754 - loss: 0.3532 - val_accuracy: 0.6538 - val_loss: 3.2936
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9725 - loss: 0.2262 - val_accuracy: 0.6538 - val_loss: 4.0526
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9631 - loss: 0.1672 - val_accuracy: 0.6538 - val_loss: 5.0893
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9694 - loss: 0.0853 - val_accuracy: 0.6538 - val_loss: 6.6376
Epoch 9/10


In [18]:
# Evaluate the model
# model.evaluate(X_test, y_test)
model.evaluate(sat_data, sat_labels)
# show trustworthiness of the model


# show trustworthiness of the model
# model.predict(X_test)
model.predict(sat_data[:1])


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9717 - loss: 0.7687 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step


array([[1.0000000e+00, 9.8277476e-11, 1.2922095e-11]], dtype=float32)

In [19]:
# Save the model
model.save("model.h5")

In [20]:
# export train and test data
np.save("train_data.npy", X_train)
np.save("train_labels.npy", y_train)
np.save("test_data.npy", X_test)
np.save("test_labels.npy", y_test)